In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import pyperclip
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import requests as req
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
  # 웹드라이버 실행
driver = webdriver.Chrome()

# 제품 목록 페이지 로드
driver.get("https://www.flipkart.com/search?sid=j9e%2Fabm%2Fhzg&otracker=CLP_Filters&p%5B%5D=facets.brand%255B%255D%3DSAMSUNG")

# 페이지 로딩 대기
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'CGtC98')))

# URL을 저장할 리스트 초기화
url_list = []

# 요소 찾기
elements = driver.find_elements(By.CLASS_NAME, 'CGtC98')

# 각 요소에서 'href' 속성을 가져와 리스트에 추가 (sponsored 요소는 제외)
for element in elements:
    try:
        # 'f8qK5m' 클래스를 가진 자식 요소가 있는지 확인
        if element.find_elements(By.CLASS_NAME, 'f8qK5m'):
            # 'f8qK5m' 클래스를 가진 자식 요소가 있다면 skip
            continue
        # 자식 요소가 'f8qK5m' 클래스를 가지고 있지 않다면 href 추출
        url_value = element.get_attribute('href')
        if url_value:  # href 속성이 있는 경우만 추가
            url_list.append(url_value)
    except NoSuchElementException:
        # 자식 요소가 없으면 그냥 넘어가기
        continue 
        
# 리뷰 데이터를 저장할 리스트 초기화
all_reviews = []

for url in tqdm(url_list):
    try:
        driver.get(url)
        
        # XPath를 사용하여 제목 찾기. 제품 코드 별 XPATH 달라서 접근 불가... 제목에서 코드 추출하세용...
        try:
            # XPath를 사용하여 제목 찾기. 제품 코드 별 XPATH 달라서 접근 불가... 제목에서 코드 추출하세용... 
            list_items = driver.find_elements(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[2]/div/div[1]/h1/span')
            
            # 텍스트 추출
            hp_data = [item.text for item in list_items]
            if not hp_data:
                hp_data = ["No data"]  # 데이터가 없을 경우 기본값
        except (NoSuchElementException, TimeoutException) as e:
            hp_data = ["Error fetching data"]  # 오류 발생 시 기본값
        
        # 리뷰 버튼 클릭 대기
        try:
            review_button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "_23J90q.RcXBOT")))
            if review_button.is_displayed():
                review_button.click()
        except (NoSuchElementException, TimeoutException):
            pass
        
        # 낮은 별점 접근
        try:
            low_rating_option = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#container > div > div.VLIitu > div > div > div.DOjaWF.gdgoEp.col-9-12 > div.DOjaWF.gdgoEp.col-12-12 > div > div:nth-child(2) > div > div > div > div > select > option:nth-child(4)")))
            low_rating_option.click()
        except (NoSuchElementException, TimeoutException) as e:
            print(f"Error selecting sort option for URL {url}: {e}")
        
        # 데이터 확보
        page_number = 1
        
        while True:
            try:
                # 페이지 URL 업데이트
                current_url = driver.current_url
                full_url = f'{current_url}&page={page_number}'
                driver.get(full_url)
                
                # ZmyHeo 클래스의 존재 여부를 확인
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ZmyHeo')))
                
                # 데이터 추출
                text_elements = driver.find_elements(By.CLASS_NAME, 'ZmyHeo')
                state_elements = driver.find_elements(By.CLASS_NAME, 'MztJPv')
                date_elements = driver.find_elements(By.CLASS_NAME, '_2NsDsF')
                star_elements = driver.find_elements(By.XPATH, "//*[contains(@class, 'XQDdHH') and (contains(@class, 'Js30Fc') or contains(@class, 'Czs3gR') or contains(@class, 'Ga3i8K'))]")
                
                text_list = [elem.text for elem in text_elements]
                state_list = [elem.text for elem in state_elements]
                date_list = [elem.text for elem in date_elements]
                dates = date_list[1::2]
                star_list = [elem.text for elem in star_elements]
                
                # 리뷰 데이터를 모든 리뷰 리스트에 추가
                for t, s, d, r in zip(text_list, state_list, dates, star_list):
                    for data in hp_data:
                        all_reviews.append({
                            'text': t,
                            'state': s,
                            'date': d,
                            'star': r,
                            'hp_data': data  # 각각의 hp_data 항목을 추가
                        })

                # 페이지 번호 증가
                page_number += 1
                
            except (NoSuchElementException, TimeoutException) as e:
                print(f"Error on page {page_number} for URL {full_url}: {e}")
                break
    
    except Exception as e:
        print(f"Error processing URL {url}: {e}")

# 현재 창 닫기
driver.quit()

  8%|██████▍                                                                            | 1/13 [00:40<08:02, 40.25s/it]

Error processing URL https://www.flipkart.com/samsung-183-l-direct-cool-single-door-4-star-refrigerator-base-drawer-digital-inverter/p/itm0e72f8c5beb0d?pid=RFRGHJPXATRDHYHF&lid=LSTRFRGHJPXATRDHYHFHFTCA4&marketplace=FLIPKART&store=j9e%2Fabm%2Fhzg&spotlightTagId=BestsellerId_j9e%2Fabm%2Fhzg&srno=b_1_4&otracker=CLP_Filters&iid=c81e135a-abb3-44e1-a771-de5421e92862.RFRGHJPXATRDHYHF.SEARCH&ssid=ooabzgb92o0000001724716351341: name 'NoSuchElementException' is not defined


 15%|████████████▊                                                                      | 2/13 [01:32<08:43, 47.56s/it]

Error processing URL https://www.flipkart.com/samsung-236-l-frost-free-double-door-3-star-convertible-refrigerator-digital-inverter-display/p/itm5d34d4279e9f1?pid=RFRGPHMKVNFYGACW&lid=LSTRFRGPHMKVNFYGACWPPYLLU&marketplace=FLIPKART&store=j9e%2Fabm%2Fhzg&srno=b_1_5&otracker=CLP_Filters&iid=c81e135a-abb3-44e1-a771-de5421e92862.RFRGPHMKVNFYGACW.SEARCH&ssid=ooabzgb92o0000001724716351341: name 'NoSuchElementException' is not defined


 23%|███████████████████▏                                                               | 3/13 [02:15<07:30, 45.08s/it]

...

Error processing URL https://www.flipkart.com/samsung-256-l-frost-free-double-door-3-star-convertible-refrigerator-digital-inverter-display/p/itmf4819f99564a9?pid=RFRGPHMKS9EFTA8Z&lid=LSTRFRGPHMKS9EFTA8ZAGBGVI&marketplace=FLIPKART&store=j9e%2Fabm%2Fhzg&srno=b_1_20&otracker=CLP_Filters&iid=c81e135a-abb3-44e1-a771-de5421e92862.RFRGPHMKS9EFTA8Z.SEARCH&ssid=ooabzgb92o0000001724716351341: name 'NoSuchElementException' is not defined


 92%|███████████████████████████████████████████████████████████████████████████▋      | 12/13 [08:13<00:38, 38.31s/it]

Error processing URL https://www.flipkart.com/samsung-183-l-direct-cool-single-door-3-star-refrigerator-digital-inverter/p/itmf01477eb08fff?pid=RFRGHJPXPYGXUFS7&lid=LSTRFRGHJPXPYGXUFS7W1JGBF&marketplace=FLIPKART&store=j9e%2Fabm%2Fhzg&srno=b_1_23&otracker=CLP_Filters&iid=c81e135a-abb3-44e1-a771-de5421e92862.RFRGHJPXPYGXUFS7.SEARCH&ssid=ooabzgb92o0000001724716351341: name 'NoSuchElementException' is not defined


100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [08:48<00:00, 40.66s/it]

Error processing URL https://www.flipkart.com/samsung-301-l-frost-free-double-door-3-star-convertible-refrigerator-digital-inverter-5-in-1-display/p/itmd9d975ab6191a?pid=RFRGPHMKUGKJHMB3&lid=LSTRFRGPHMKUGKJHMB3INDZGL&marketplace=FLIPKART&store=j9e%2Fabm%2Fhzg&srno=b_1_24&otracker=CLP_Filters&iid=c81e135a-abb3-44e1-a771-de5421e92862.RFRGPHMKUGKJHMB3.SEARCH&ssid=ooabzgb92o0000001724716351341: name 'NoSuchElementException' is not defined


In [3]:
df_SS_refri = pd.DataFrame(all_reviews)
df_SS_refri

,text,state,date,star,hp_data
0,Water is coming,"Certified Buyer, Chirala",6 days ago,1,SAMSUNG 183 L Direct Cool Single Door 4 Star R...
1,Side body over when use lo cooling ... please ...,"Certified Buyer, Anakapalli District",21 days ago,1,SAMSUNG 183 L Direct Cool Single Door 4 Star R...
2,Not cooling fast bad quality... Im not happy,"Certified Buyer, Samastipur District",27 days ago,1,SAMSUNG 183 L Direct Cool Single Door 4 Star R...
3,"Not up to the mark, it is producing too much s...","Certified Buyer, Presidency Division",1 month ago,1,SAMSUNG 183 L Direct Cool Single Door 4 Star R...
4,Worst product,"Certified Buyer, Bathinda",1 month ago,1,SAMSUNG 183 L Direct Cool Single Door 4 Star R...
...,...,...,...,...,...
1375,Best ......,"Certified Buyer, Jamnagar",7 months ago,4,SAMSUNG 301 L Frost Free Double Door 3 Star Co...
1376,Well works good since its new and good capacit...,"Certified Buyer, Chennai",9 months ago,4,SAMSUNG 301 L Frost Free Double Door 3 Star Co...
1377,Best products at this price range also getting...,"Certified Buyer, Nagpur",9 months ago,4,SAMSUNG 301 L Frost Free Double Door 3 Star Co...
1378,i got it with exchange of old refrigerator.\nl...,"Certified Buyer, Muktsar","Apr, 2023",4,SAMSUNG 301 L Frost Free Double Door 3 Star Co...


In [4]:
df_SS_refri.to_csv("df_SS_refri.csv")